<h1><center>Assignment 2</center></h1>
<h4><center>Pin-Jie Lin</center></h4>
<center>pili00001@stud.uni-saarland.de</center>
<center>https://github.com/pjlintw/CL20/tree/main/assignment2</center>

## 1 Implementation of POS tagging with HMMs

We implement all necessary functions including unknown words dealing under a class `HiddenMarkovTagger`.
, which is implemented as a Bigram-based HMM tagger and can be tranined and evaluated on CoNLL datasets. The tagger estimates an optimal sequence of tags by using Viterbi algorithm `_viterbi_searching()`. The model uses de-utb/de-train.tt for traning, evaluate its performance on file `de-utb/de-test.t`. Modify the file path if you are not using the same dataset.

### 1.1 Parameters and Viterbi of HiddenMarkovTagger

**HMM parameters.** The main parameters of HMM tagger consist of three probabilities, namly **initial probability**, **transition probability** and **emission probability**. Since we estimate a bigram-based tagger, we can compute the probabilities with frequencies of tag, word and the transition. The abstract formula of probabilities are 

                    initital probability   : num_state_i_at_idx_0  / num_sents
                    transition probability : num_state_i2j         / num_state_i
                    emission probability   : num_state_token_ij    / num_state_i
                    
We compute the frequencies of the training set first. Then creating three probabilities sparately.

<img src="./img/carbon.png" width="600"><center>Figure 1. Create Counter to record the frequencies of the token, tag, token at first position, the transition and the emission </center>

**Structure of HiddenMarkovTagger** The class has one main method `.train()` for training and evaluting. The steps for using the class are constructing the class and call its `.train()` method. We can specify the evaluation by passing `True` to `do_eval` parameter. The method will automatically evaluate the on the test file given from `eval_file`. 

<img src="./img/SS3.png" width="600"><center>Figure 2. The structure of `HiddenMarkovTagger` </center>

## 2. Running the HMM tagger

To train the tagger and evaluate on test file. Simply execute `main.py` in command line.

> python main.py

<img src="./img/SS1.png" width="600"><center>Figure 5. It shows the size of probabilities matrix and execution time for training `HiddenMarkovTagger`.</center>

## 3. Dealing with unknown words (extra credit)

In our implementation, we design 4 mechanisms to deal the missing token in vocabulary while inference.

**Assumption.** Since we've learnd to produce the next token using N-gram model by given most recent context. We made an assumption that **the missing tokens must follow certain previous words**. Therefore, we replace to the unknown word by replacing a generated token from bigram model trained on the training set, which make sure the generated token is in the vocabulary.

**Model design** Based on the assumption, we design 2 models using generated token replacing methods. The **HMM(bigram)**, which is a baseline, uses bigram model to replace the unknown word by predicting the missing one in vocabulary given one previous word. In order to produce the missing token at first position, we add a start of sentence token `[SOS]` to vocabulary and the training corpus for bigram model. Whenever encountering a unknown word in the beginning of sentence, we fit the `[SOS]` to generate first token for replacing.

Since the generated token for the first position have always the same previous context `[SOS]`. We want the first generated token to be generated in more random way. The **HMM(bigram+randomFirstToken)** tries to improve the unknown word at first position by random sampling first words from all first words in training sets.

After the two methods, we considers a more general way for dealing unknown word. The third one **HMM(sampleDistribution)** simply replaces all unknown word by **sampling a token from word distribution** of the training sets. Finally, **HMM(empirical)** follows empirical rule of unknown word, which usually are a `NOUN`. We replace all unknown words by a word whose tag is NOUN.

**Configurations** To execute various HMMtagger, we have to adjust the parameters of HiddenMarkovTagger (Line 335). 
There are three parameters for controlling which mechanisums will be used, `use_bigram`, `sample_from_distribution`  and `random_first_token`. To use different functions for unknown word, pass bool to the parameters of the class like the table beloq. Notice that: random_first_token will be used when use_bigram is True.

```python
# Line 330 in main.py
hmm_tagger = HiddenMarkovTagger(data_dir=DATADIR,
                                train_file=train_file,
                                eval_file=test_file,
                                use_bigram=False,
                                sample_from_distribution=False, 
                                random_first_token=False)
```

**Experimental Result.** We made the smiliar previous context assumption for first two models
**HMM(bigram)** and **HMM(bigram+randomFirstToken)**. The bigram model for generating unknown token gives us excellent score for accuracy. When we try to replace the first unknown word by random sampling. It can get one more score. We are gussing that the probability of generating the first token from training corpus and test corpos are not exactly smilar. Therefore, random sampling can perform better.

We also try a general way to replace the unknown word. The method of sampling from distribution get 3 more points than baseline. But the execution is pretty slow. We then apply the prior knowledge of unknown words for the mehtod by simply replacing every unknown word with a token `Kosten`, whose tag is `NOUN`. It shown that the **HMM(empirical) is the fastest and outperformer the others.

<img src="./img/SS2.png" width="1000"><center>Figure 5. Various `HiddenMarkovTagger` and its performance.</center>